In [1]:
# Goal:visualize ensembles
# are semi supervised networks better calibrated?
# Step 1: visualize ensemble
%load_ext autoreload
%autoreload 2
# now visualize 3D reconstruction

In [2]:
# given the ensembles, now average proportional to likelihood and average the likelihoods to get one final trace
# is this trace significantly better than other traces?
# it is hard to measure calibration because the likelihoods are wrt sigmoid and not softmax
# first load the ensembles and get trace estimates
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from tensorflow.python.util import deprecation
deprecation._PRINT_DEPRECATION_WARNINGS = False
from deepgraphpose.models.fitdgp_util import get_snapshot_path
import os
os.environ['DLClight'] = 'True'
from deepgraphpose.models.eval import load_pose_from_dlc_to_dict, estimate_pose
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib
matplotlib.use('Agg')
from moviepy.editor import VideoFileClip
from matplotlib import animation
import numpy as np
#video_name = 'left_movieflip'
video_name = 'right_movie'

task="iblright"
#full_dir = '/datahd2a/datasets/tracki/{}'.format(task)
#full_dir = '/data/datasets/tracki' #/{}'.format(task)
task="iblfingers"
full_dir = "/share/home/ekb2154/data/datasets/tracki/{}/model_data/".format(task)
#task="iblright"
scorer="kelly"
# new dates for ensemble
ranges = range(1,5)
dates= ["2030-01-0{}".format(ii) for ii in ranges]
run_names= ["run {}".format(ii) for ii in ranges]
frame_range = np.arange(0, 1000).astype('int')
n_frames = frame_range.size
# video_name = 'ibl60left'  #['ibl60left', 'ibl60right']
moviefile = "/home/ekb2154/data/libraries/dgp_paninski/etc/ensembles/iblvideos/" + video_name + '.mp4'#project_dir / "videos" / ("{}.mp4".format(video_name))
snapshot ='snapshot-step3-final--0'
outname = '{}'.format(task)
#figname = 'comparison_{}.mp4'.format(outname)
shuffle=1
# make plot comparing them
xrs, yrs, likelihoods, heatmaps = [], [], [], []
for date_ in dates:
    project_dir = Path(full_dir)/ "{}-{}-{}".format(task,scorer, date_)
    print("\n{}\n".format(project_dir))
    video_files = os.listdir(project_dir / "videos_pred" )
    video_files = [video_file_ for video_file_ in video_files if '.csv' in video_file_]
    #video_files = [video_file_ for video_file_ in video_files if video_name in video_file_]
    #import pdb; pdb.set_trace()
    #assert  len(video_files) == 1
    #video_files =['{}{}.csv'.format(video_name, snapshot)]
    print(video_files)
    video_files =['{}_labeled{}.csv'.format(video_name, snapshot)]
    #break
    label_file = str(project_dir / "videos_pred" / video_files[0])
    #label_file = project_dir / "videos_pred" / ("{}_labeled{}.csv".format(video_name, snapshot))
    # load labels
    print(label_file.rsplit('/',1)[-1])
    # if we don't need heatmaps
    #labels = np.load(label_file, allow_pickle=True)[()]
    labels = load_pose_from_dlc_to_dict(label_file)
    # since we need heatmaps
    #snapshot_path, _ = get_snapshot_path(snapshot, project_dir, shuffle=shuffle)
    #cfg = auxiliaryfunctions.read_config(cfg_yaml)
    #labels = estimate_pose( str(project_dir/ "config.yaml"), snapshot_path, moviefile, "", shuffle=shuffle,
    #    save_pose=False, save_str='', new_size=None, save_heatmap=True)
    xr = labels['x']  # T x D
    yr = labels['y']  # T x D
    likelihood = labels['likelihoods']
    #heatmap = labels['heatmap']
    xr = xr[frame_range, :]
    yr = yr[frame_range, :]
    likelihood=likelihood[frame_range,:]
    xrs.append(xr)
    yrs.append(yr)
    likelihoods.append(likelihood)
    #heatmaps.append(heatmap)

DLC loaded in light mode; you cannot use any GUI (labeling, relabeling and standalone GUI)

/share/home/ekb2154/data/datasets/tracki/iblfingers/model_data/iblfingers-kelly-2030-01-01

['left_movieflip_labeledsnapshot-step3--1998.csv', 'right_movie_labeledsnapshot-step3-final--0.csv', 'left_movieflip_labeledsnapshot-step3-final--0.csv', 'right_movie_labeledsnapshot-step3--1998.csv']
right_movie_labeledsnapshot-step3-final--0.csv

/share/home/ekb2154/data/datasets/tracki/iblfingers/model_data/iblfingers-kelly-2030-01-02

['right_movie_labeledsnapshot-step3-final--0.csv', 'left_movieflip_labeledsnapshot-step3--2664.csv', 'right_movie_labeledsnapshot-step3--2664.csv', 'left_movieflip_labeledsnapshot-step3-final--0.csv']
right_movie_labeledsnapshot-step3-final--0.csv

/share/home/ekb2154/data/datasets/tracki/iblfingers/model_data/iblfingers-kelly-2030-01-03

['left_movieflip_labeledsnapshot-step3--2331.csv', 'right_movie_labeledsnapshot-step3-final--0.csv', 'left_movieflip_labeledsnapshot-s

In [3]:
# Make video from disagreements
num_frames, nj = xrs[0].shape
num_ensembles = len(xrs)
# how would you plot disagreement between frames
xrs_mean = np.stack(xrs,0).mean(0)
yrs_mean = np.stack(yrs,0).mean(0)
def distance_mean(xr, yr, xrm, yrm):
    distance = np.sqrt((xr- xrm)**2 + (yr - yrm)**2)
    return distance

dmean = np.zeros((num_ensembles, num_frames, nj))
for ii, (xr, yr)  in enumerate(zip(xrs, yrs)):
    dmean[ii] = distance_mean(xr, yr, xrs_mean, yrs_mean)


In [4]:
distance_threshold = 40
mask_outliers = dmean > distance_threshold
mask_outliers = mask_outliers.sum(0).sum(1)

In [13]:
frame_indices = np.argwhere(mask_outliers).flatten()

In [16]:
frame_indices= np.argsort(dmean.sum(0).sum(1))[::-1][:len(frame_indices)+5]

In [17]:
# frames where these networks disagree
from moviepy.editor import VideoFileClip, VideoClip, ImageSequenceClip
clip = VideoFileClip(moviefile)
fps = clip.fps
ny, nx = clip.size
num_new_frames = len(frame_indices)
randomly_sampled_frames = np.zeros((num_new_frames, nx,ny, 3))
for idxs, frame_idx in enumerate(frame_indices):
        randomly_sampled_frames[idxs] = clip.get_frame(frame_idx *1.0/fps)#[:,:, 0]
clip2 = ImageSequenceClip(list(randomly_sampled_frames), fps=fps)
clip2_name =Path(moviefile).parent / (Path(moviefile).stem + "_diff.mp4")
clip2.write_videofile(str(clip2_name))
clip.close()
clip2.close()
print('Created disagreement video in {}'.format(str(clip2_name)))

t:   0%|          | 0/26 [00:00<?, ?it/s, now=None]

Moviepy - Building video /home/ekb2154/data/libraries/dgp_paninski/etc/ensembles/iblvideos/right_movie_diff.mp4.
Moviepy - Writing video /home/ekb2154/data/libraries/dgp_paninski/etc/ensembles/iblvideos/right_movie_diff.mp4



Moviepy - Done !
Moviepy - video ready /home/ekb2154/data/libraries/dgp_paninski/etc/ensembles/iblvideos/right_movie_diff.mp4
Created disagreement video in /home/ekb2154/data/libraries/dgp_paninski/etc/ensembles/iblvideos/right_movie_diff.mp4


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
plt.hist(dmean[0])
plt.show()

In [ ]:
%matplotlib inline
"""
tplot = 0
njplot = 0 
fig, ax = plt.subplots(nensembles+1, 1)
for ii in range(nensembles):
    ax[ii].imshow(heatmaps[ii][tplot,:,:, njplot])
ax[-1].imshow(median_hmap[tplot,:,:, njplot])

plt.show()
# From this median read new coordinates 
coordinates = np.zeros((nt, nj))
for ntt in range(nt):
    for njj in range(nj):
        coordinates[ntt, njj]= np.argmax()
"""

In [ ]:
nt, nj = xr.shape

In [ ]:
x_wmean = np.empty_like(xr)
y_wmean = np.empty_like(yr)
l_wmean = np.empty_like(yr)#
l_weight = np.dstack(likelihoods).sum(-1)

num_runs = len(xrs)
for ii in range(num_runs):
    x_wmean += xrs[ii]*likelihoods[ii]
    y_wmean += yrs[ii]*likelihoods[ii]
    l_wmean += likelihoods[ii]*likelihoods[ii]

x_wmean /=l_weight
y_wmean /=l_weight
l_wmean/=l_weight

In [ ]:
likelihoods2= np.dstack(likelihoods)
max_cte = np.zeros((nt, nj))

for tt in range(nt):
    for njj in range(nj):
        max_cte[tt, njj]= np.argmax(likelihoods2[tt, njj])
max_cte = max_cte.astype('int')

In [ ]:
x_wmean = np.empty_like(xr)
y_wmean = np.empty_like(yr)
for tt in range(nt):
    for njj in range(nj):
        xte_  = max_cte[tt,njj]
        x_wmean[tt,njj] = xrs[xte_][tt,njj]
        y_wmean[tt,njj] = yrs[xte_][tt,njj]

In [ ]:
#y_wmean[0,0]
run_names.append('ensemble')
xrs.append(x_wmean)
yrs.append(y_wmean)

In [ ]:
nj = xr.shape[1]
num_traces = len(xrs)
bodyparts = ['part {}'.format(ii) for ii in range(nj)] # replace w real name
#%%
print('Init video')
video_clip = VideoFileClip(str(moviefile))
fps = video_clip.fps
n_frames = min(n_frames, np.ceil(video_clip.fps * video_clip.duration).astype('int'))
frame_init = frame_range[0] / fps
frame_stop = frame_range[-1] / fps
# video_clip = video_clip.subclip(0, (n_frames / fps)) # it didnt take into account the range
video_clip = video_clip.subclip(frame_init, frame_stop) # it didnt take into account the range

In [ ]:
#np.unravel_index(np.argmax(x_wmean), x_wmean.shape)

In [ ]:
xrs[0].max()

In [ ]:
num_rows

In [ ]:
#%%
from itertools import chain
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

mpl.rcParams.update({  # 'font.sans-serif' : 'Helveltica',
    'axes.labelsize': 9,
    'xtick.labelsize': 9,
    'ytick.labelsize': 9,
    'axes.titlesize': 11})
#%%
# coment out y 
num_coordinates = 1 # x and y or just 2
fig2 = plt.figure(constrained_layout=False, figsize=(11, 10))
widths = [2, 7]
heights = np.ones(num_coordinates * nj)
num_cols = 2
num_rows = num_coordinates*nj
spec2 = fig2.add_gridspec(ncols=num_cols, nrows=num_rows, width_ratios=widths,
                          height_ratios=heights)

framerows = num_rows// num_traces
#%% Plot
markersize = 2
markersize2 = 5
from matplotlib.lines import Line2D
color_class = plt.cm.ScalarMappable(cmap="cool")
colors = color_class.to_rgba(np.linspace(0, 1, nj))
markers = Line2D.filled_markers[:num_traces]

#%%
frame_0name = 0
frame = video_clip.get_frame(0)
tidx = 0
colors_traces= ['C{}'.format(ii) for ii in range(num_traces)]

imagesfig = [[]]*num_traces
imagesmakers = [[]]*num_traces*nj
xtracesmarkers = [[]]*num_traces*nj
ytracesmarkers = [[]]*num_traces*nj
frametitles = [[]]*num_traces

count= 0
count1 = 0
for ss_idx in range(num_traces):
    # for the image
    f2_ax1 = fig2.add_subplot(spec2[ss_idx*framerows:ss_idx*framerows+ framerows, 0])
    if ss_idx == 0:
        frametitles[ss_idx] = f2_ax1.set_title('Frame {} \n {} '.format(frame_0name, run_names[ss_idx]),
                                               color=colors_traces[ss_idx], weight='bold')
    else:
        frametitles[ss_idx] = f2_ax1.set_title('{}'.format(run_names[ss_idx]),
                                               color=colors_traces[ss_idx], weight='bold')
    imagesfig[ss_idx] = f2_ax1.imshow(frame)
    dgp_frames = []
    for njj in range(nj):
        dxs_dgp = xrs[ss_idx][tidx]
        dys_dgp = yrs[ss_idx][tidx]
        dgp_frame, = f2_ax1.plot(dxs_dgp[njj], dys_dgp[njj], c=colors[njj],
            marker=markers[ss_idx], ms=markersize, )
        imagesmakers[count] = dgp_frame
        count+=1
    # x traces
    for njj in range(nj):
        f2_ax2 = fig2.add_subplot(spec2[num_coordinates * njj, 1:])
        f2_ax2.set_yticks([])
        f2_ax2.set_xticks([])
        # plot trace
        f2_ax2.plot(xrs[ss_idx][:, njj], c=colors_traces[ss_idx], linewidth=1.5, linestyle='--')
        dgp_mtx, = f2_ax2.plot(tidx, dxs_dgp[njj], c=colors_traces[ss_idx], marker=markers[ss_idx], markersize=markersize2,
                               markerfacecolor=None)
        #f2_ax2.set_ylabel('x')
        f2_ax2.set_xlim([0, n_frames])
        f2_ax2.set_ylabel('{}'.format(bodyparts[njj]), color=colors[njj], weight='bold')

        xtracesmarkers[count1] = dgp_mtx
        count1+=1

allmyparts = imagesfig + imagesmakers + xtracesmarkers
    
plt.tight_layout()
plt.show()

In [ ]:
xrs[0].shape

In [ ]:
# is this the best possible outcome?
# need to visualize a video w masked likelihoods
labels = {}
labels['likelihoods']= 

In [ ]:
video_clip = VideoFileClip(str(video_file))
mask_array = labels['likelihoods'].T > mask_threshold
# make movie
create_annotated_movie(
        video_clip,
        labels['x'].T,
        labels['y'].T,
        mask_array=mask_array,
        filename=save_file, dotsize=dotsize, colormap=colormap)

video_clip.close()

In [ ]:
#heatmaps are too expensive